In [126]:
import pandas as pd
import numpy as np
import datetime as dt

xlsx = pd.ExcelFile("Data5_23_Correct.xlsx", engine = "openpyxl")
dataOEE_Page = pd.read_excel(xlsx,'Twin Screw OEE')
dataTS_Page = pd.read_excel(xlsx, "Twin Screw ")
dateInterval = 1

In [127]:
def startUp():
    #This section of code takes filler sections of the excel spreadsheet from both tabs and removes them

    global dataOEE_Page
    global dataTS_Page

    dataOEE_Page.replace('x', np.nan, inplace= True)
    dataTS_Page.replace('x', np.nan, inplace = True)

    dataOEE_Page.drop(labels = 0, axis = 0, inplace = True)
    dataTS_Page.drop(labels = 0, axis = 0, inplace = True)

    #print(dataOEE_Page)

    #Names the indexes for the columns of the dataframes

    dataOEE_Page.columns = ['Date', 'Lbs Produced', '%OEE', 'Labor Hours', 'Lb/Man Hour', 'Pails Filled', 'Pails/Man Hour', 'Removal', 'Week of', 'Weekly %OEE']
    dataTS_Page.columns = ['Date', 'Num of Ops', 'Total Minutes', 'Maintenance', 'Making Powder', 'Making Prepolymer', 'Startup', 'Breaks', 'LBS Produced', 'Pails Filled', 'Minutes Worked', 'Efficiency', 'Unnamed', 'Unnamed', 'Unnamed']

    #dataOEE_Page

    #This section trims the fat off the data set and removes entries that are effectively empty from both the OEE and Efficiency
    #tab. 

    dataTS_Page = dataTS_Page.dropna(how = 'all', axis = 1)
    dataTS_Page = dataTS_Page.dropna(subset=['Num of Ops'])

    dataOEE_Page = dataOEE_Page.dropna(subset=['Labor Hours'])
    dataOEE_Page = dataOEE_Page.dropna(how = 'all', axis = 1)

    #Assigns type data to the values in the columns, certain columns are uninterpretable due to the extra data within them 

    dataOEE_Page = dataOEE_Page.astype({"Lbs Produced": np.float64, "%OEE": np.float64, "Labor Hours": np.float64, "Lb/Man Hour": np.float64, "Pails Filled": np.float64, "Pails/Man Hour": np.float64, 'Weekly %OEE': np.float64})
    dataTS_Page = dataTS_Page.astype({'Num of Ops': np.float64, 'Total Minutes': np.float64, 'Maintenance': np.float64, 'Efficiency': np.float64})

    #This section will identify the percent values that are based as a decimal in the updated dataframes
    #From here the new data is passed down onto future methods for stronger correlations

    dataOEE_Page.loc[dataOEE_Page['%OEE'] < 1, '%OEE'] = round(dataOEE_Page['%OEE']*100, 0)
    dataTS_Page.loc[dataTS_Page['Efficiency'] < 1, 'Efficiency'] = round(dataTS_Page['Efficiency']*100, 0)



In [128]:
def pairDateData(OEEdata, TSdata):
    #This section of code takes the first 13 entries of the Twin Screw OEE tab of the excel sheet and removes them from the data pool
    #This effectively removes the logic error that occurred due to the lack of the # of operators on the - tab, and realigns the dates
    #Following this section, data that doesn't have a matching pair datewise will be removed from each respective dataframe

    OEEdata.drop(dataOEE_Page.index[range(0,13)], axis = 0, inplace =True)

    OEEdata = OEEdata.reset_index(drop = True)
    TSdata = TSdata.reset_index(drop = True)

    #Using the dates from the trimmed lists, then comparing them to make a condensed list of data points
    #This resultant list should contain all eligible data points
    #From this comparison dataframe we should be able to pull data that correlates directly
    #Should only be done in order to compare data cross sheet due to lack of variables

    dateCheck = [OEEdata['Date'], TSdata['Date']] #Makes a list of the columns from either dataFrame
    dateCheck = pd.concat(dateCheck, axis=1, ignore_index=False, verify_integrity=False) #Combines the dataCheck list into a dataFrame
    dateCheck['Removal_OEE'] = np.nan #Adds columns of nan values
    dateCheck['Removal_TS'] = np.nan 
    dateCheck.columns = ['OEE_Date', 'TS_Date','Removal_OEE','Removal_TS'] #Renames the columns of the dataframe

    #Checks whether or not the dates correspond to each other throughout either column, result is recorded in respective removal column
    #Recorded as True if there is matching pair, False if there isn't
    dateCheck['Removal_OEE'] = (dateCheck['OEE_Date'].isin(dateCheck['TS_Date']))
    dateCheck['Removal_TS'] = (dateCheck['TS_Date'].isin(dateCheck['OEE_Date']))

    OEEindex = dateCheck[dateCheck['Removal_OEE'] == False].index #Identifies the exact index of the date cell needing removed
    TSindex = dateCheck[dateCheck['Removal_TS'] == False].index

    #print(OEEindex)
    #print(TSindex)

    #Drops erroneous cells from both data sets
    OEEdata = OEEdata.drop(OEEindex, inplace=False)
    TSdata = TSdata.drop(TSindex,inplace=False)

    OEEdata = OEEdata.reset_index(drop = True)
    TSdata = TSdata.reset_index(drop = True)
    TSdata = TSdata.drop(labels = ['Date'], axis = 1)

    combo = pd.concat([OEEdata, TSdata], axis = 1, ignore_index = False)
    return combo


In [129]:
def narrowDates(df, date):

    #Will act as test parameters to prepare method for true use
    testDay = True #Will be the program default
    testWeek = False
    testMonth = False
    testDate = pd.Timestamp(date + " 00:00:00")
    daysInMonth = testDate.daysinmonth
    beginningOfInterval = pd.Timestamp()
    print(daysInMonth)
    userInput = 'Tier 3'

    
    df['Date'] = pd.to_datetime(df['Date'])

    #Basically decides how far back the date retrieval will go.
    global dateInterval
    if (userInput == "Tier 2"):
        beginningOfInterval = testDate
    elif (userInput == 'Tier 3'):
        beginningOfInterval = testDate - pd.Timedelta(str(dateInterval) + ' day')
    elif (userInput == 'Tier 5'): #If not month's end it will retrieve all prior data up to that point
        beginningOfInterval = testDate.replace(day=1)
    else:
        print('System Error: Invalid selection')

    if (testDate in df['Date'].values):
        print ("It's here")
    else:
        print('Not in dataset')


    #Will set index of the dataframe to that of the dates to be easily moved through and sorted by
    df.set_index('Date', inplace=True)
    return (df['Date'].loc[beginningOfInterval:testDate])





In [130]:
startUp()
pairedDate = pairDateData(dataOEE_Page, dataTS_Page)
passedDate = "2021-05-17"
narrowDates(pairedDate, passedDate)

31
It's here
2021-05-10 00:00:00
